In [9]:
import sqlite3
from prettytable import PrettyTable

conn = sqlite3.connect('example.db')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS City
             (Id INTEGER PRIMARY KEY, Name TEXT UNIQUE)''')

c.execute('''CREATE TABLE IF NOT EXISTS Person 
             (Name TEXT, CityId INTEGER, 
              FOREIGN KEY(CityId) REFERENCES City(Id) ON DELETE SET NULL)''')


c.execute("DELETE FROM Person")
c.execute("DELETE FROM City")

c.executemany("INSERT OR REPLACE INTO City (Id, Name) VALUES (?, ?)", [
    (1, 'Москва'),
    (2, 'Санкт-Петербург'),
    (3, 'Казань')
])

c.executemany("INSERT INTO Person (Name, CityId) VALUES (?, ?)", [
    ('Андрей', 1),
    ('Леонид', 2),
    ('Сергей', 1),
    ('Григорий', 4)  
])

conn.commit()


def display_table(title, headers, rows):
    table = PrettyTable()
    table.title = title
    table.field_names = headers
    for row in rows:
        table.add_row([row[0] if row[0] is not None else "NULL", row[1] if row[1] is not None else "NULL"])
    print(table)
    print()


print("INNER JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             INNER JOIN City
             ON Person.CityId = City.Id''')
display_table("INNER JOIN", ["Person", "City"], c.fetchall())

print("LEFT OUTER JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             LEFT OUTER JOIN City
             ON Person.CityId = City.Id''')
display_table("LEFT OUTER JOIN", ["Person", "City"], c.fetchall())

print("RIGHT OUTER JOIN (эмулируется с помощью LEFT JOIN):")
c.execute('''SELECT Person.Name, City.Name
             FROM City
             LEFT OUTER JOIN Person
             ON Person.CityId = City.Id
             ORDER BY City.Name, Person.Name''')
display_table("RIGHT OUTER JOIN", ["Person", "City"], c.fetchall())

print("FULL OUTER JOIN (эмулируется с помощью LEFT JOIN и UNION):")
c.execute('''
    SELECT Person.Name, City.Name
    FROM Person
    LEFT OUTER JOIN City ON Person.CityId = City.Id
    UNION ALL
    SELECT Person.Name, City.Name
    FROM City
    LEFT OUTER JOIN Person ON Person.CityId = City.Id
    WHERE Person.CityId IS NULL
    ORDER BY 2, 1
''')
display_table("FULL OUTER JOIN", ["Person", "City"], c.fetchall())

print("LEFT JOIN с проверкой на NULL:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             LEFT JOIN City
             ON Person.CityId = City.Id
             WHERE City.Id IS NULL''')
display_table("LEFT JOIN с NULL", ["Person", "City"], c.fetchall())

print("RIGHT JOIN с проверкой на NULL (эмулируется с помощью LEFT JOIN):")
c.execute('''SELECT Person.Name, City.Name
             FROM City
             LEFT JOIN Person
             ON Person.CityId = City.Id
             WHERE Person.CityId IS NULL
             ORDER BY City.Name''')
display_table("RIGHT JOIN с NULL", ["Person", "City"], c.fetchall())

conn.close()


INNER JOIN:
+--------------------------+
|        INNER JOIN        |
+--------+-----------------+
| Person |       City      |
+--------+-----------------+
| Андрей |      Москва     |
| Леонид | Санкт-Петербург |
| Сергей |      Москва     |
+--------+-----------------+

LEFT OUTER JOIN:
+----------------------------+
|      LEFT OUTER JOIN       |
+----------+-----------------+
|  Person  |       City      |
+----------+-----------------+
|  Андрей  |      Москва     |
|  Леонид  | Санкт-Петербург |
|  Сергей  |      Москва     |
| Григорий |       NULL      |
+----------+-----------------+

RIGHT OUTER JOIN (эмулируется с помощью LEFT JOIN):
+--------------------------+
|     RIGHT OUTER JOIN     |
+--------+-----------------+
| Person |       City      |
+--------+-----------------+
|  NULL  |      Казань     |
| Андрей |      Москва     |
| Сергей |      Москва     |
| Леонид | Санкт-Петербург |
+--------+-----------------+

FULL OUTER JOIN (эмулируется с помощью LEFT JOIN и UNIO